# Cleaning Data Exercises

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.




## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_cleaning.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [90]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

survey = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [91]:
acs = survey[["age", "sex", "educ", "inctot"]]
acs.sample(10)

,age,sex,educ,inctot
152623,58,male,4 years of college,150000
178023,50,female,4 years of college,92000
86655,65,male,4 years of college,38400
312287,46,female,5+ years of college,46000
212121,53,male,grade 12,56000
224684,58,female,4 years of college,37100
29926,45,male,grade 12,60000
92347,34,female,2 years of college,32000
62005,78,female,1 year of college,12000
287274,45,male,grade 12,800


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [92]:
import numpy as np

acs["inctot"] = acs["inctot"].replace(9_999_999, np.nan)
acs.sample(10)

,age,sex,educ,inctot
307065,34,male,1 year of college,20000.0
213955,31,male,grade 12,26600.0
32347,39,male,5+ years of college,638000.0
29537,26,male,4 years of college,38000.0
2725,31,female,1 year of college,22900.0
170506,10,female,nursery school to grade 4,NaN
1621,45,male,grade 12,0.0
289765,77,male,grade 12,59250.0
239161,64,female,2 years of college,74900.0
280406,56,male,5+ years of college,73000.0


### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [ ]:
# acs['age'].mean()

TypeError: 'Categorical' with dtype category does not support reduction 'mean'

The error we see is: 
TypeError: 'Categorical' with dtype category does not support reduction 'mean'.
This means that the data type is categorical, which is not the expected float data type we need for the age variable for calculation.

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

In [94]:
results = {}
print(acs["age"].dtype)
ex5_age_young = "less than 1 year old"
ex5_age_old = "90 (90+ in 1980 and 1990)"

results["ex5_age_young"] = ex5_age_young
results["ex5_age_old"] = ex5_age_old

acs["age"].value_counts()

category


age
60    4950
54    4821
56    4776
59    4776
58    4734
      ... 
93     476
95     471
92     355
91     227
96      10
Name: count, Length: 97, dtype: int64

### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [95]:
acs["age"] = acs["age"].cat.rename_categories({ex5_age_young: "0", ex5_age_old: "90"})

### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [96]:
acs["age"] = pd.to_numeric(acs["age"])

ex7_avg_age = acs["age"].mean()

results["ex7_avg_age"] = ex7_avg_age

print(f"The average age of the dataset is {ex7_avg_age:.0f}")

The average age of the dataset is 41


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [97]:
over18 = acs[acs["age"] >= 18]

ex8_avg_age = over18["age"].mean()

results["ex8_avg_age"] = ex8_avg_age

print(f"The average age of people over 18 is {ex8_avg_age:.0f}")

The average age of people over 18 is 50


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [98]:
over18["educ"].value_counts()

over18["college_degree"] = (over18["educ"] == "4 years of college") | (over18["educ"] == "5+ years of college")

# no of people with college degree

college_grads = over18.loc[over18["college_degree"] == True]
ex9_num_college = len(college_grads)
results["ex9_num_college"] = ex9_num_college

print(f"The number of people in the dataset with a college degree is {ex9_num_college:,.0f}")

The number of people in the dataset with a college degree is 77,013


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [99]:
pd.crosstab(over18['sex'], over18["college_degree"])

college_degree,False,True
sex,,
male,85821,36181
female,90200,40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [100]:
cross = pd.crosstab(over18["sex"], over18["college_degree"], normalize="index")
print(cross)
ex11_share_male_w_degrees = cross.loc["male", True]
ex11_share_female_w_degrees = cross.loc["female", True]

results["ex11_share_male_w_degrees"] = ex11_share_male_w_degrees
results["ex11_share_female_w_degrees"] = ex11_share_female_w_degrees
print(f"The share of men with college degrees is {ex11_share_male_w_degrees:.3f}")
print(f"The share of women with college degrees is {ex11_share_female_w_degrees:.3f}")

college_degree     False     True 
sex                               
male            0.703439  0.296561
female          0.688381  0.311619
The share of men with college degrees is 0.297
The share of women with college degrees is 0.312


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [101]:
over_40 = over18[over18["age"] >= 40]
under_40 = over18[over18["age"] < 40]

cross_under40 = pd.crosstab(
    under_40["sex"], under_40["college_degree"], normalize="index"
)
print(cross_under40)
cross_over40 = pd.crosstab(over_40["sex"], over_40["college_degree"], normalize="index")
print(cross_over40)

diff_over40 = abs(cross_over40.loc["male", True] - cross_over40.loc["female", True])
diff_under40 = abs(cross_under40.loc["male", True] - cross_under40.loc["female", True])

print(diff_under40)
print(diff_over40)

if diff_over40 > diff_under40:
    ex12_comparing = "the absolute difference has increased"
else:
    ex12_comparing = "the absolute difference has decreased"

print(ex12_comparing)

results["ex12_comparing"] = ex12_comparing

college_degree     False     True 
sex                               
male            0.743143  0.256857
female          0.665710  0.334290
college_degree     False     True 
sex                               
male            0.682123  0.317877
female          0.699144  0.300856
0.07743230514952698
0.017020152658337562
the absolute difference has decreased


### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).

As can be seen from the difference noted above, the absolute difference in college attainment levels have decreased. The absolute difference in older cohort was 7.7% in favor of women. Where as in younger cohort (under 40), this difference has narrowed to 1.7% points, slightly favoring men. The levels for men have seen significant jump and speaks to the shift in educational patterns in which there is gender parity in college attainment in younger people than older people.

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 

In [ ]:
# Educational attainment levels between white americans, black americans, hispanic americans, asian americans, and native americans

dfnew = survey[["age", "race", "hispan","sex", "educ", "inctot"]]

dfnew["inctot"] = dfnew["inctot"].replace(9_999_999, np.nan)

dfnew["age"] = dfnew["age"].cat.rename_categories({ex5_age_young: "0", ex5_age_old: "90"})

dfnew["age"] = pd.to_numeric(dfnew["age"])

dfnew = dfnew[dfnew["age"] >= 18]

dfnew["college_degree"] = (dfnew["educ"] == "4 years of college") | (dfnew["educ"] == "5+ years of college")

# White Non Hispanic Educational Attainment Levels
white_non_hispanic = dfnew[(dfnew["race"] == "white") & (dfnew["hispan"] == "not hispanic")]

white_non_hispanic_cross = pd.crosstab(white_non_hispanic['race'], white_non_hispanic["college_degree"], normalize="index")

white_non_hispanic_cross

# White Hispanic Educational Attainment Levels
white_hispanic = dfnew[(dfnew["race"] == "white") & (dfnew["hispan"] != "not hispanic")]

white_hispanic_cross = pd.crosstab(white_hispanic['race'], white_hispanic["college_degree"], normalize="index")

white_hispanic_cross

# Black Non Hispanic Educational Attainment Levels

black_non_hispanic = dfnew[(dfnew["race"] == "black/african american/negro") & (dfnew["hispan"] == "not hispanic")]

black_non_hispanic_cross = pd.crosstab(black_non_hispanic['race'], black_non_hispanic["college_degree"], normalize="index")

black_non_hispanic_cross


# Black Hispanic Educational Attainment Levels

black_hispanic = dfnew[(dfnew["race"] == "black/african american/negro") & (dfnew["hispan"] != "not hispanic")]

black_hispanic_cross = pd.crosstab(black_hispanic['race'], black_hispanic["college_degree"], normalize="index")

black_hispanic_cross


# Asian Non Hispanic Educational Attainment Levels

asian_non_hispanic = dfnew[(dfnew["race"] == "other asian or pacific islander") & (dfnew["hispan"] == "not hispanic")]

asian_non_hispanic_cross = pd.crosstab(asian_non_hispanic['race'], asian_non_hispanic["college_degree"], normalize="index")

asian_non_hispanic_cross


# Asian Hispanic Educational Attainment Levels

asian_hispanic = dfnew[(dfnew['race'] == "other asian or pacific islander") & (dfnew["hispan"] != "not hispanic")]

asian_hispanic_cross = pd.crosstab(asian_hispanic['race'], asian_hispanic["college_degree"], normalize="index")

asian_hispanic_cross


print(
    f"The share of white non hispanic with college degrees is {white_non_hispanic_cross.loc['white', True]:.3f} "
    f"where as the share of black non hispanic with college degrees is {black_non_hispanic_cross.loc['black/african american/negro', True]:.3f}.\n"
    f"The share of white hispanics with college degrees is {white_hispanic_cross.loc['white', True]:.3f} "
    f"while the share of black hispanics with college degree is {black_hispanic_cross.loc['black/african american/negro', True]:.3f}.\n"
    f"The share of asian hispanics with college degrees is {asian_hispanic_cross.loc['other asian or pacific islander', True]:.3f} "
    f"while the share of asian non hispanics with college degrees is {asian_non_hispanic_cross.loc['other asian or pacific islander', True]:.3f}"
)





The share of white non hispanic with college degrees is 0.335 where as the share of black non hispanic with college degrees is 0.192.
The share of white hispanics with college degrees is 0.173 while the share of black hispanics with college degree is 0.173.
The share of asian hispanics with college degrees is 0.226 while the share of asian non hispanics with college degrees is 0.492
